# OpenAI API 사전 준비

In [ ]:
# 패키지 설치
!pip install openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
# 환경변수 준비
import os
os.environ["OPENAI_API_KEY"] = ""

# 학습 데이터 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# 데이터 세트 준비
df = pd.read_csv(
    'tsukuyomi1.csv',
    usecols=[1,2],
    names=['prompt','completion'],
    skiprows=2)
df.to_json(
    "tsukuyomi.jsonl",
    orient='records',
    lines=True,
    force_ascii=False
)

In [ ]:
# 데이터 세트 검증
!openai tools fine_tunes.prepare_data \
    -f tsukuyomi.jsonl \
    -q

Analyzing...

- Your file contains 432 prompt-completion pairs
- More than a third of your `prompt` column/key is uppercase. Uppercase prompts tends to perform worse than a mixture of case encountered in normal language. We recommend to lower case the data if that makes sense in your domain. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion 

In [ ]:
! pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.6 MB/s eta 0:00:00


In [ ]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

data_path = "tsukuyomi.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["prompt"]:
    print(message)

Num examples: 432
First example:
배
가
 
고
프
다


In [ ]:
# dataset[0]

In [ ]:
# # Format error checks
# format_errors = defaultdict(int)

# for ex in dataset:
#     if not isinstance(ex, dict):
#         format_errors["data_type"] += 1
#         continue

#     messages = ex.get("messages", None)
#     if not messages:
#         format_errors["missing_messages_list"] += 1
#         continue

#     for message in messages:
#         if "role" not in message or "content" not in message:
#             format_errors["message_missing_key"] += 1

#         if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
#             format_errors["message_unrecognized_key"] += 1

#         if message.get("role", None) not in ("system", "user", "assistant", "function"):
#             format_errors["unrecognized_role"] += 1

#         content = message.get("content", None)
#         function_call = message.get("function_call", None)

#         if (not content and not function_call) or not isinstance(content, str):
#             format_errors["missing_content"] += 1

#     if not any(message.get("role", None) == "assistant" for message in messages):
#         format_errors["example_missing_assistant_message"] += 1

# if format_errors:
#     print("Found errors:")
#     for k, v in format_errors.items():
#         print(f"{k}: {v}")
# else:
#     print("No errors found")

# 파인튜닝 실행

In [ ]:
# # 파인튜닝 수행
# !openai api fine_tunes.create \
#     -t "tsukuyomi_prepared.jsonl" \
#     -m davinci

from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("tsukuyomi_prepared.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-aXYQP9CuJueXPLcsxd09b5H8', bytes=64228, created_at=1719299196, filename='tsukuyomi.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
# # 파인튜닝 작업 상태 확인
# !openai api fine_tunes.get \
#     -i ft-ElEoXDioUUTHHm3lxUmYIr2D

In [ ]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
  training_file="file-P9uyfMG1AvVJ1VkW9v1Ao9cV",
  model="davinci-002"
)


FineTuningJob(id='ftjob-Ky4XNjL8x3hutiqKq2L5R6U2', created_at=1719299211, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-PLIHvIZaN5qBCns42JU7An4M', result_files=[], seed=1191957944, status='validating_files', trained_tokens=None, training_file='file-aXYQP9CuJueXPLcsxd09b5H8', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# # List 10 fine-tuning jobs
# # client.fine_tuning.jobs.list(limit=10)

# # # Retrieve the state of a fine-tune
# # client.fine_tuning.jobs.retrieve("ftjob-abc123")

# # # Cancel a job
# # client.fine_tuning.jobs.cancel("ftjob-abc123")

# # List up to 10 events from a fine-tuning job
# client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-x6ulc4zBad6Q9QuckoiS2HgR", limit=10)

# # Delete a fine-tuned model (must be an owner of the org the model was created in)
# # client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

In [ ]:
# !curl https://api.openai.com/v1/fine_tuning/jobs/ft-AF1WoRqd3aJAHsqc9NY7iL8F \
#   -H "Authorization: Bearer $OPENAI_API_KEY"


# 추론 실행

In [ ]:
# import openai

# # 추론 실행
prompt="좋아하는 음식은 무엇인가요? ->"
# response = openai.Completion.create(
#     model="ft:gpt-3.5-turbo-0125",
#     prompt=prompt,
#     max_tokens=100,
#     stop="\n")
# print(response["choices"][0]["text"])

from openai import OpenAI
client = OpenAI()

completion = openai.completions.create(
  model="ft:davinci-002:personal::9duGTZF0",
  prompt=prompt,
  max_tokens=100,
  stop="\n"
)
# print(completion["text"])
# print(completion["choices"])
# print(completion.choices[0])
print(completion.choices[0].text)


 그림에 그린 떡입니다!


# 파인튜닝 모델 목록 조회 및 삭제

In [ ]:
# 파인튜닝 모델 목록 보기
!openai api fine_tunes.list

In [ ]:
# 파인튜닝 모델 삭제
!openai api models.delete \
    -i <모델 ID>